In [2]:
import os
import pandas as pd

### change the current working directory
os.chdir('/home/jovyan/mixtures/example_sims/')

### read in the data
#ev = pd.read_csv("Afr_CEU_sas_eas_10000tot_2500Afr_2500sas_2500eas_sims_and_reference.txt", sep='\t')
ev = pd.read_csv("eur_CEU_sas_eas_10000tot_1500eur_2500CEU_2500sas_3500eas_sims_and_reference.txt", sep='\t')


ev.head(5) ### look at the first 5 rows

,CHR,SNP,eur_MAF,CEU_MAF,sas_MAF,eas_MAF,af
0,1,rs1001834,0.398521,0.3737,0.399800,0.391850,0.38940
1,1,rs1004650,0.039602,0.0303,0.109408,0.165687,0.09775
2,1,rs10127461,0.170796,0.2071,0.171792,0.218240,0.19875
3,1,rs10127554,0.413364,0.4293,0.264823,0.073413,0.26130
4,1,rs10127801,0.069306,0.0404,0.000000,0.000000,0.01985


In [15]:
A = ev.filter(['eur_MAF','CEU_MAF','sas_MAF', 'eas_MAF'], axis=1) # Take only columns with ancestry Minor Allele Frequency
af = ev.filter(['af']) # Take the column with Total Allele Frequency
guess = [60, .10, .10, .10] # Set up a starting guess.

[60, 0.1, 0.1, 0.1]


In [16]:
### Hidden Ancestries Script

### HA : (A, taf, x_guess) -> (x_answer, n_iterations, time)

### A generalized function that takes 3 inputs: 
 
 ## 1. Genetic data in a matrix "A" size Nxk containing N SNPs (these are the rows), and k ancestries (these are the columns);

 ## 2. The total allele frequency called "taf" which should be an Nx1 vector
 
 ## 3. A starting guess "x_guess" which should be a kx1 vector
    
### and returns 3 outputs:

 ## 1. The hidden proportions of every ancestry in the data as a kx1 vector
    
 ## 2. The number of iterations that SLSQP did as a scalar value

 ## 3. The run time of the algoirthm as a scalar value, measured in seconds

import numpy as np
import scipy as scipy
from scipy.optimize import minimize
import timeit

def HA(A, taf, x_guess):
    
    # Grab the number of ancestries
    k=np.shape(A)[1]
    
    # Start the clock!
    start = timeit.default_timer()
    
    # This is the objective function!
    def obj_fun(x):
        b=0
        for i in range(0,k):
            b=b + x[i]*A[:,i:(i+1)]
        b=b-taf
        return np.sum(b**2, axis=0)[0]
    
    # This is the gradient of the objective function!
    def grad_obj_fun(x):

        gradvec = np.zeros((k,1))

        d=0

        for i in range(0,k):
            d=d + x[i]*A[:,i:(i+1)]
        d=d-taf

        for i in range(0,k):
            gradvec[i,:] = np.sum(2*A[:,i:(i+1)]*d, axis=0)
        return gradvec

    # These are wrappers that make our constraints and our bounds

    cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x,axis=0) -1},)

    for i in range(0,k-1):
        cons = cons + ({'type': 'ineq', 'fun': lambda x: x[i]},)

    bnds = ((0, None), (0, None))

    for i in range(0,k-1):
        bnds = bnds + ((0, None),)

    ans_obj = scipy.optimize.minimize(obj_fun, x_guess, method='SLSQP', jac=grad_obj_fun, bounds=bnds, constraints=cons, tol=1e-5)
    
    stop = timeit.default_timer()
    
    time= stop-start
    
    return ans_obj.x, ans_obj.nit, time

In [17]:
HA(A, af, guess)

IndexError: SLSQP Error: the length of bounds is not compatible with that of x0.